In [7]:
import pickle 
import numpy as np
import torch
from sklearn.manifold.t_sne import trustworthiness

In [2]:
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from util.plot_embedding import plot_embedding, plot_embedding_subplot

cuda not available


In [3]:
task_parameters = {}
task_parameters['flag_task'] = 'clustering'
task_parameters['reduction_method'] = 'spectral'
task_parameters['n_components'] = 2

net_parameters = {}
net_parameters['D'] = 784
net_parameters['n_components'] = task_parameters['n_components']
net_parameters['H'] = 50
net_parameters['L'] = 10

In [9]:
filename = 'data/set_100_mnist_spectral_size_200_500.pkl'
with open(filename, 'rb') as f:
    [all_test_data] = pickle.load(f)

In [13]:
# Loss functions
def l2_norm(v):
    return np.sqrt(np.sum(np.square(v), axis=1, dtype=np.float64))

def pairwise_loss_function_1(y_true, y_pred, W):
    pairwise_loss_1 = mean_squared_error(y_true[W.row,:], y_true[W.col,:])
    pairwise_loss_2 = mean_squared_error(y_pred[W.row,:], y_pred[W.col,:])
    return np.square(pairwise_loss_1 - pairwise_loss_2)

def pairwise_loss_function_2(y_true, y_pred, W):
    pairwise_loss_1 = l2_norm(y_true[W.row,:] - y_true[W.col,:])
    pairwise_loss_2 = l2_norm(y_pred[W.row,:] - y_pred[W.col,:])
    return np.average(np.square(pairwise_loss_1 - pairwise_loss_2))

In [21]:
def evaluate_net(net, all_test_data):
    n_test = len(all_test_data)
    similarity_tracker = np.zeros((n_test, 1))
    trust_tracker = np.zeros((n_test, 2))
    
    for i in range(n_test):
        G = all_test_data[i]
        y_pred = net.forward(G).detach().numpy()
        similarity_tracker[i,0] =  pairwise_loss_function_2(G.target, y_pred, G.adj_matrix)
        trust_tracker[i,0] = trustworthiness(G.signal, G.target, n_neighbors=5)
        trust_tracker[i,1] = trustworthiness(G.signal, y_pred, n_neighbors=5)
    return similarity_tracker, trust_tracker

In [22]:
# Load trained net
net = GraphConvNet2(net_parameters, task_parameters)
root = 'results/mnist_spectral2/'
filename = root + 'net5.pkl'
checkpoint = torch.load(filename)
net.load_state_dict(checkpoint['state_dict'])x

similarity_tracker, trust_tracker = evaluate_net(net, all_test_data)

In [24]:
print(np.average(similarity_tracker, axis=0))
print(np.average(trust_tracker, axis=0))

[0.05158281]
[0.7597092  0.70382048]


In [27]:
# Load trained net
net = GraphConvNet2(net_parameters, task_parameters)
root = 'results/mnist_spectral2/'
filename = root + 'net1.pkl'
checkpoint = torch.load(filename)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker2, trust_tracker2 = evaluate_net(net, all_test_data)


nb of hidden layers= 10
dim of layers (w/ embed dim)= [784, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]




In [28]:
print(np.average(similarity_tracker2, axis=0))
print(np.average(trust_tracker2, axis=0))

[0.06258942]
[0.7597092  0.69042917]
